# Simple genetic algorithm for TSP

This is simple GA for TSPs from TSPLIB (http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsplib.html). 

Things to do:
- add new selection operators: tournament without return, roulette and rank selection 
- add new mutation operators: RSM, PSM and RSM/PSM hybrid
- add new cross-ver operators: EAX, Edge-3, OX2, POS, CX2
- add methods of population evaluation: population diversity, convergence rate
- apply elitism after end evaluation (10% of the worst individuals generated are replaced by the best current parents)

In [90]:
import random
from datetime import datetime
import os

## Additional functions

### Dynamic parameters

In [91]:
def dynamic_params_1(param):
    param_list = None
    if type(param) == list:
        param_list = param
        param = param_list[0]
    return param, param_list

def dynamic_params_2(param, params_list, i, num_of_iterations, to_int):
    if params_list != None:
        if params_list[0] < params_list[1]: 
            param = params_list[0] + (i/num_of_iterations) * (params_list[1] - params_list[0])
        else: 
            param = params_list[0] - (i/num_of_iterations) * (params_list[0] - params_list[1])
    if to_int:
        return int(round(param, 0))
    else:
        return param

### Euc2D to matrix transformation

In [92]:
def EUC_2D_to_matrix(file_path):    
    file_list = open(file_path).readlines()
    file_name = file_path.split('/')[-1].split('.')[0]
    x_list, y_list = [], []
    for line in file_list:
        if line.split()[0].isdigit():
            node, x, y = line.split()
            x_list.append(float(x))
            y_list.append(float(y))
        elif line.split()[0] == 'EOF':
            break

    fhand = open('data/matrices/' + file_name, 'a')
    fhand.write(str(len(x_list)) + '\n')
    for i in range(len(x_list)):
        for j in range(len(x_list)):
            xd = x_list[i] - x_list[j]
            yd = y_list[i] - y_list[j]
            dist_ij = int(round((xd**2 + yd**2)**0.5, 0))
            fhand.write(str(dist_ij) + ' ')
            if i == j:
                break
        fhand.write('\n')
    fhand.close()

# path_name = 'data/coords'
# files_listed = [fname for fname in os.listdir(path_name) if os.path.isfile(os.path.join(path_name, fname))]
# results = None
# for fname in files_listed:
#     EUC_2D_to_matrix(path_name + '/' + fname)

### Load distance matrix

In [93]:
def calculate_distance_matrix(file_name):
    lines = open(file_name).readlines()
    num_of_cities = int(lines[0].strip())
    matrix = [[0 for _ in range(num_of_cities)] for _ in range(num_of_cities)]
    
    i_line = 0
    for line in lines[1:]: 
        row = list(map(int, line.split())) 
        for i in range(len(row)):
            matrix[i_line][i] = row[i]
            matrix[i][i_line] = row[i]
        i_line += 1
    return matrix

### Display stats

In [94]:
def the_best_solution(population): 
    return min(population,key=lambda ind: ind[1])

def avg_solutions(population): 
    return round(sum([ind[1] for ind in population])/len(population))

## Create new population

In [95]:
def new_individual(n): 
    individual = [i for i in range(n)]
    random.shuffle(individual)
    return individual

def initialize_population(m, n): 
    return [[new_individual(n), -1] for _ in range(m)]

## Fitness function

In [96]:
def calculate_fitness(individual, d_matrix):
    sum_dist = 0
    ind = individual[0] 
    for i in range(len(ind)-1): 
        sum_dist += d_matrix[ind[i]][ind[i+1]] 
    
    sum_dist += d_matrix[ind[-1]][ind[0]] 

    individual[1] = sum_dist 

def evaluate_population(population, distance_matrix):
    for individual in population:
        calculate_fitness(individual, distance_matrix)

## Genetic operators

### Selection

#### Tournament selection with return

In [97]:
def select_individual(population, k):
    rand_index = random.randint(0,len(population)-1)
    min_individual = population[rand_index]

    for _ in range(k-1):
        rand_index = random.randint(0,len(population)-1)
    if min_individual[1] > population[rand_index][1]:
        min_individual = population[rand_index]

    return min_individual

def selection_tournament(population, k):
    new_pop = [] 

    for _ in range(len(population)):
        sel_ind = select_individual(population, k)
        copy_sel_ind = [sel_ind[0][:],sel_ind[1]]
        new_pop.append(copy_sel_ind)

    return new_pop

### Mutation

#### Inversion mutation

In [98]:
def mutation_inversion(population, probability): 
    for individual in population:
        if random.random() < probability:
            gene = individual[0]
            cutting_point_1 = random.randint(0,len(gene)-1)
            cutting_point_2 = random.randint(cutting_point_1+1,len(gene))
            gene[cutting_point_1:cutting_point_2] = gene[cutting_point_1:cutting_point_2][::-1]

#### Exchange mutation

In [99]:
def mutation_exchange(population, probability):
    for individual in population:
        if random.random() < probability:
            gene = individual[0]
            index_1 = random.randint(0,len(gene)-1)
            index_2 = random.randint(0,len(gene)-1)
            if index_1 != index_2:
                gene[index_1], gene[index_2] = gene[index_2], gene[index_1]

#### Displacement mutation (with probability of inversion)

In [100]:
def mutation_displacement(population, probability): 
    for individual in population:
        if random.random() < probability: 
            gene = individual[0]
            cutting_point_1 = random.randint(0, len(gene)-1)
            cutting_point_2 = random.randint(cutting_point_1+1, len(gene))
            inversion = random.randint(0, 1)
            if inversion:
                gene_piece = gene[cutting_point_1:cutting_point_2][::-1]
            else:
                gene_piece = gene[cutting_point_1:cutting_point_2]

            gene_temp = gene[:cutting_point_1] + gene[cutting_point_2:]
            insert_point = random.randint(0, len(gene_temp))
            if insert_point == cutting_point_1:
                insert_point -= 1
                
            gene = gene_temp[:insert_point] + gene_piece + gene_temp[insert_point:]
            individual[0] = gene

#### Scramble mutation

In [101]:
def mutation_scramble(population, probability):
    for individual in population:
        if random.random()<probability: 
            gene = individual[0]
            cp_1 = random.randint(0,len(gene)-3)
            cp_2 = random.randint(cp_1+3,len(gene))
            gene_slice = gene[cp_1:cp_2]
            random.shuffle(gene_slice)
            gene[cp_1:cp_2] = gene_slice

#### SBM (Select Best Mutation)

In [102]:
def mutation_select_best(population, probability, distance_matrix):
    for individual in population:
        if random.random() < probability:
            ind_IM = individual.copy()
            ind_EM = individual.copy()
            ind_DM = individual.copy()
            ind_SM = individual.copy()

            results = {}

            mutation_inversion([ind_IM], 1)
            calculate_fitness(ind_IM, distance_matrix)
            results[ind_IM[1]] = ind_IM

            mutation_exchange([ind_EM], 1)
            calculate_fitness(ind_EM, distance_matrix)
            results[ind_EM[1]] = ind_EM

            mutation_displacement([ind_DM], 1)
            calculate_fitness(ind_DM, distance_matrix)
            results[ind_DM[1]] = ind_DM

            mutation_scramble([ind_SM], 1)
            calculate_fitness(ind_SM, distance_matrix)
            results[ind_SM[1]] = ind_SM

            min_key = min(results, key = results.get)
            individual = results[min_key]

#### SAM (Select Any Mutation)

In [103]:
def mutation_select_any(population, probability, distance_matrix):
    for individual in population:
        if random.random() < probability:
            mut_op = random.choice(['IM','EM','DM','SM'])
            if mut_op == 'IM':
                mutation_inversion([individual], 1)
            elif mut_op == 'EM':
                mutation_exchange([individual], 1)
            elif mut_op == 'DM':
                mutation_displacement([individual], 1)
            elif mut_op == 'SM':
                mutation_scramble([individual], 1)

### Cross-over

#### PMX (Partially-Mapped Crossover)

In [104]:
def gene_swap_PMX(parent, child, second_child, start, end, cp_1, cp_2):
    for i in range(start,end): 
        parent_gene = parent[i] 
        while parent_gene in child[cp_1:cp_2]:
            index = child.index(parent_gene)
            parent_gene = second_child[index]
        child[i] = parent_gene

def crossover_PMX(population, probability):
    new_population = []
    for i in range(0,len(population),2): 
        parent_1 = population[i][0] 
        parent_2 = population[i+1][0] 

    if random.random()<probability: 
        cp_1 = random.randint(1,len(parent_1)-3)
        cp_2 = random.randint(cp_1+1,len(parent_1)-1)

        child_1 = [None for _ in range(len(parent_1))]
        child_2 = child_1[:] 

        child_1[cp_1:cp_2] = parent_2[cp_1:cp_2]
        child_2[cp_1:cp_2] = parent_1[cp_1:cp_2]

        gene_swap_PMX(parent_1, child_1, child_2,0,cp_1,cp_1,cp_2) 
        gene_swap_PMX(parent_1, child_1, child_2,cp_2,len(parent_1),cp_1,cp_2) 
        gene_swap_PMX(parent_2, child_2, child_1,0,cp_1,cp_1,cp_2) 
        gene_swap_PMX(parent_2, child_2, child_1,cp_2,len(parent_1),cp_1,cp_2)

        new_population.append([child_1, -1])
        new_population.append([child_2, -1])

    else: 
        new_population.append([parent_1,-1])
        new_population.append([parent_2,-1])

    return new_population

#### OX (Order Crossover) 

In [105]:
def generate_child_OX(parent_1, parent_2, cp_1, cp_2):
    sequence = parent_1[cp_2:] + parent_1[:cp_2] 
    gene_temp = []
    for item in sequence:
        if item not in parent_2[cp_1:cp_2]:
            gene_temp.append(item)
    return gene_temp[:cp_1] + parent_2[cp_1:cp_2] + gene_temp[cp_1:]

def crossover_OX(population, probability):
    new_population = []
    for i in range(0,len(population),2):
        parent_1 = population[i][0] 
        parent_2 = population[i+1][0] 

        if random.random()<probability: 
            cp_1 = random.randint(1,len(parent_1)-3)
            cp_2 = random.randint(cp_1+1,len(parent_1)-1)

            child_1 = generate_child_OX(parent_1, parent_2, cp_1, cp_2)
            child_2 = generate_child_OX(parent_2, parent_1, cp_1, cp_2)

            new_population.append([child_1, -1])
            new_population.append([child_2, -1])

        else: 
            new_population.append([parent_1,-1])
            new_population.append([parent_2,-1])

    return new_population

#### CX (Cycle Crossover) 

In [106]:
def crossover_CX(population, probability):
    def create_cycle(parent_1, parent_2):
        # first_el = random.choice([parent_1[0], parent_2[0]]) # optional random choice of parents
        first_el = parent_1[0]
        index = 0 
        next_el = None
        cycle_1, cycle_2 = [], []
        while next_el != first_el:
            el_p1, el_p2 = parent_1[index], parent_2[index]
            cycle_1.append(el_p1)
            cycle_2.append(el_p2)
            index = parent_1.index(el_p2)
            next_el = el_p2
        return cycle_1, cycle_2
         
    def generate_child_CX(parent_1, parent_2, cycle_1, cycle_2):
        def create_child(parent, cycle_1, cycle_2):
            child = [None for i in range(len(parent))]
            for i in range(len(parent)):
                if parent[i] in cycle_1:
                    child[i] = cycle_2[cycle_1.index(parent[i])]
            for i in range(len(parent)):
                if child[i] == None:
                    child[i] = parent[i]
            return child
        child_1 = create_child(parent_1, cycle_1, cycle_2)
        child_2 = create_child(parent_2, cycle_2, cycle_1)
        return child_1, child_2

    new_population = []
    for i in range(0,len(population),2): 
        parent_1 = population[i][0] 
        parent_2 = population[i+1][0] 

        if random.random() < probability: 
            cycle_1, cycle_2 = create_cycle(parent_1, parent_2)

            child_1, child_2 = generate_child_CX(parent_1, parent_2, cycle_1, cycle_2)

            new_population.append([child_1, -1])
            new_population.append([child_2, -1])

        else: 
            new_population.append([parent_1,-1])
            new_population.append([parent_2,-1])

    return new_population

#### ER (Edge Recombination Crossover)

In [107]:
def create_edge_map(parent_1, parent_2):
    def neighbour_list(parent):
        n_dict = {} 
        for i in range(len(parent)-1):
            n_dict[parent[i]] = [parent[i-1], parent[i+1]]
        n_dict[parent[-1]] = [parent[-2], parent[0]]
        return n_dict
            
    n_dict_1 = neighbour_list(parent_1)
    n_dict_2 = neighbour_list(parent_2)

    result = {}
    for key, value in n_dict_1.items():
        result[key] = list(set(n_dict_2[key] + value))
    return result

def generate_child_ER(parent_1, parent_2, edge_map):
    def filter_edge_map(edge_map, current_city):
        for key, value in edge_map.items():
            if current_city in value:
                value.remove(current_city)
                edge_map[key] = value
        return edge_map

    def find_next_city(edge_map, current_city):
        cities = edge_map[current_city]
        temp_dict = {key : edge_map[key] for key in cities}
        shortest = min(len(item) for item in temp_dict.values())
        shortest_list = [key for key, value in temp_dict.items() if len(value) == shortest] 
        return random.choice(shortest_list)

    local_edge_map = edge_map.copy()
    sequence = []

    cities_1 = edge_map[parent_1[0]]
    cities_2 = edge_map[parent_2[0]]
    if len(cities_1) > len(cities_2):
        current_city = parent_1[0]
    elif len(cities_2) > len(cities_1):
        current_city = parent_2[0]
    else:
        current_city = random.choice([parent_1[0], parent_2[0]])
    sequence.append(current_city)
    local_edge_map = filter_edge_map(local_edge_map, current_city)

    for i in range(len(parent_1)-1):
        try:
            current_city = find_next_city(local_edge_map, current_city)
        except:
            current_city = random.choice(list(set(parent_1) - set(sequence)))
        sequence.append(current_city)
        local_edge_map = filter_edge_map(local_edge_map, current_city)

    return sequence

def crossover_ER(population, probability):
    new_population = []
    for i in range(0,len(population),2): 
        parent_1 = population[i][0] 
        parent_2 = population[i+1][0] 

        if random.random()<probability: 
            edge_map = create_edge_map(parent_1, parent_2)
            child_1 = generate_child_ER(parent_1, parent_2, edge_map)
            child_2 = generate_child_ER(parent_2, parent_1, edge_map)

            new_population.append([child_1, -1])
            new_population.append([child_2, -1])

        else: 
            new_population.append([parent_1,-1])
            new_population.append([parent_2,-1])

    return new_population

#### Edge-2

In [108]:
# creating edge map (with additional mechanism preserving common subsequences)
def create_edge_map_2(parent_1, parent_2):
    def neighbour_list(parent):
        n_dict = {} 
        for i in range(len(parent)-1):
            n_dict[parent[i]] = [parent[i-1], parent[i+1]]
        n_dict[parent[-1]] = [parent[-2], parent[0]]
        return n_dict
            
    n_dict_1 = neighbour_list(parent_1)
    n_dict_2 = neighbour_list(parent_2)

    result = {}
    for key, value in n_dict_1.items():
        val_with_dup = n_dict_2[key] + value
        val_without_dup = []
        for val in val_with_dup:
            if val in val_without_dup:
                val_without_dup.remove(val)
                val_without_dup.append(-abs(val))
            else:
                val_without_dup.append(val)
        result[key] = val_without_dup
    return result

def generate_child_E2(parent_1, parent_2, edge_map):
    def filter_edge_map(edge_map, current_city):
        for key, value in edge_map.items():
            value_abs = []
            for item in value:
                value_abs.append(abs(item))
            if current_city in value_abs:
                try:
                    value.remove(current_city)
                except:
                    value.remove(-current_city)
                edge_map[key] = value
        return edge_map

    def find_next_city(edge_map, current_city):
        if current_city == None:
            result = random.choice(list(set(parent_1) - set(sequence)))
        else:
            negative_value = []
            for value in edge_map[current_city]:
                if value < 0:
                    negative_value.append(abs(value))
            if len(negative_value) == 1 and len(edge_map[current_city]) > 1:
                result = negative_value[0]
            else:
                cities_dict = {abs(key) : edge_map[abs(key)] for key in edge_map[current_city]}
                cities_lens = [len(item) for item in cities_dict.values()]
                if cities_lens:
                    shortest = min(cities_lens)
                    shortest_keys = [key for key, value in cities_dict.items() if len(value) == shortest]
                    result = random.choice(shortest_keys)
                else:
                    result = random.choice(list(set(parent_1) - set(sequence)))
        return result 

    local_edge_map = edge_map.copy()
    sequence = []

    current_city = random.choice(parent_1)
    sequence.append(current_city)
    local_edge_map = filter_edge_map(local_edge_map, current_city)
   
    for i in range(len(parent_1)-1): 
        current_city = find_next_city(local_edge_map, current_city)
        sequence.append(current_city)
        local_edge_map = filter_edge_map(local_edge_map, current_city)
    return sequence

def crossover_E2(population, probability):
    new_population = []
    for i in range(0,len(population),2): 
        parent_1 = population[i][0]
        parent_2 = population[i+1][0] 

        if random.random()<probability: 
            edge_map = create_edge_map_2(parent_1, parent_2)
            child_1 = generate_child_E2(parent_1, parent_2, edge_map)
            child_2 = generate_child_E2(parent_2, parent_1, edge_map)

            new_population.append([child_1, -1])
            new_population.append([child_2, -1])

        else: 
            new_population.append([parent_1,-1])
            new_population.append([parent_2,-1])

    return new_population

## Optional 2-opt

In [109]:
def calculate_distance(route, cp_1, cp_2, d_matrix):
    if cp_1 > cp_2:
        cp_1, cp_2 = cp_2, cp_1
    dist_1 = d_matrix[route[cp_1 - 1]][route[cp_1]]
    dist_2 = d_matrix[route[cp_2 - 1]][route[cp_2]]
   
    return dist_1 + dist_2
 
def two_opt(population, probability, d_matrix, k): 
    for individual in population:
        if random.random() < probability:
            route = individual[0]
            for i in range(k):
                for cp_1 in range(len(route) - 2):
                    for cp_2 in range(cp_1 + 2, len(route)):
                        old_distance = calculate_distance(route, cp_1, cp_2, d_matrix)
                        new_route = route.copy()
                        new_route[cp_1:cp_2] = new_route[cp_1:cp_2][::-1]
                        new_distance = calculate_distance(new_route, cp_1, cp_2, d_matrix)
                        if new_distance < old_distance: 
                            route = new_route
            individual[0] = route

## Program execution 

In [110]:
def execute_GA(
    file_path, 
    num_of_ind, 
    num_of_iterations, 
    crossover,
    mutation, 
    k,
    p_m, 
    p_c,
    p_2opt = False,
    k_2opt = False
    ):

    # start time measurement
    start_time = datetime.now()
    dt_string = start_time.strftime("%d/%m/%Y %H:%M:%S")
    dt_string_short = start_time.strftime("%d%m%y_%H%M%S")

    matrix = calculate_distance_matrix(file_path)
    num_of_cities = len(matrix) 
    population = initialize_population(num_of_ind, num_of_cities)
    evaluate_population(population, matrix)
    best_solution = the_best_solution(population)

    # optional dynamic parameters
    dynamic_p_m = False
    if p_m == 'dynamic':
        p_m = 0.01
        dynamic_p_m = True

    k, k_list = dynamic_params_1(k)
    p_m, p_m_list = dynamic_params_1(p_m)
    p_c, p_c_list = dynamic_params_1(p_c)

    tsp_name = file_path.split('/')[-1].split('.')[0]

    fhand = open('results/'+tsp_name+'_'+dt_string_short+'.txt', 'a')
    fhand.write(dt_string)
    fhand.write('\n\nTSP file: {}\n'.format(tsp_name))
    fhand.write('Population size: {}\n'.format(num_of_ind))
    fhand.write('Number of iterations: {}\n'.format(num_of_iterations))
    fhand.write('Crossover operator: {}\n'.format(crossover))
    fhand.write('Mutation operator: {}\n'.format(mutation))
    fhand.write('2-opt (p_opt/k_opt): {}/{}\n\n'.format(p_2opt, k_2opt))
    fhand.write('{:>9}\t{:>9}\t{:>9}\t{:>9}\t{:>9}\t{:>9}\t{:>9}\t{:>9}\n'.format('iter', 'time', 'best_sol', 
                                                                           'avg', 'p_m', 'p_c', 'k', 'best_gen'))
    
    stagnation_alarm = 0

    for i in range(num_of_iterations+1):
        # selection
        population = selection_tournament(population, k)

        # cross-over
        if crossover == 'PMX':
            population = crossover_PMX(population, p_c) 
        elif crossover == 'OX':
            population = crossover_OX(population, p_c) 
        elif crossover == 'CX':
            population = crossover_CX(population, p_c)
        elif crossover == 'ER':
            population = crossover_ER(population, p_c)
        elif crossover == 'E2':
            population = crossover_E2(population, p_c)

        # simple anti-stagnation mechanism
        if dynamic_p_m and stagnation_alarm > num_of_iterations/20:
            p_m = p_m * 1.5
            stagnation_alarm = 0

        # mutation
        if mutation == 'IM':
            mutation_inversion(population, p_m)
        elif mutation == 'EM':
            mutation_exchange(population, p_m)
        elif mutation == 'DM':
            mutation_displacement(population, p_m)
        elif mutation == 'SM':
            mutation_scramble(population, p_m)
        elif mutation == 'SBM':
            mutation_select_best(population, p_m, matrix)
        elif mutation == 'SAM':
            mutation_select_any(population, p_m, matrix)

        # 2-opt
        if p_2opt and k_2opt:
            two_opt(population, p_2opt, matrix, k_2opt)

        evaluate_population(population, matrix)
        local_best_solution = the_best_solution(population)
        if best_solution[1] <= local_best_solution[1]:
            stagnation_alarm += 1
        elif best_solution[1] > local_best_solution[1]: 
            best_solution = local_best_solution
            stagnation_alarm = 0

        if i%(num_of_iterations/100) == 0:
            time = round((datetime.now() - start_time).total_seconds(), 2)
            print('iter: {}, time: {}, best: {}, avg: {}'.format(i, time, best_solution[1], avg_solutions(population)))
            fhand.write('{:>9}\t{:>9}\t{:>9}\t{:>9}\t{:>9}\t{:>9}\t{:>9}\t{:>9}\n'.format(i, time, best_solution[1], avg_solutions(population), 
                                                                                      round(p_m, 2), round(p_c, 2), k, str(best_solution[0])))
        
        k = dynamic_params_2(k, k_list, i, num_of_iterations, True) 
        p_m = dynamic_params_2(p_m, p_m_list, i, num_of_iterations, False)
        p_c = dynamic_params_2(p_c, p_c_list, i, num_of_iterations, False) 

        if stagnation_alarm > 3000: # or time > 600:
            break                                                               

    # execution time measurement
    td = datetime.now() - start_time
    minutes = td.total_seconds()//60
    seconds = td.total_seconds() - minutes*60
    fhand.write('\nExecution time: {:.0f} min {:.2f} sec'.format(minutes, seconds))
    fhand.close()

    # displaying the summary
    print('TSP:', tsp_name)
    print('cross_op = {}, mut_op = {}, p_m_end = {}, p_c_end = {}, k_end = {}'.format(
            crossover, mutation, round(p_m, 2), round(p_c, 2), k))
    print('Solution:', best_solution[1])
    print('Execution time: {:.0f} min {:.2f} sec'.format(minutes, seconds))
    print('----------')

    # reset czasu
    start_time = datetime.now()

In [111]:
path_name = 'data/matrices/'
file_names = [fname for fname in os.listdir(path_name) if os.path.isfile(os.path.join(path_name, fname))]

for fname in file_names:
    execute_GA(
        file_path = path_name + fname,
        num_of_ind = 100, 
        num_of_iterations = 10000,
        crossover = 'OX',
        mutation = 'DM',
        k = 3, # selection pressure
        p_m = [0, 0.2], # mutation probability
        p_c = 0.8, # cross-over probability
        # optional 2-opt params
        # p_2opt = 0.005, 
        # k_2opt = 1
    )

iter: 0, time: 0.17, best: 31825, avg: 33699
iter: 100, time: 8.75, best: 24534, avg: 26200
iter: 200, time: 17.71, best: 21045, avg: 22592
iter: 300, time: 28.01, best: 20677, avg: 22793
iter: 400, time: 41.94, best: 20677, avg: 22620
iter: 500, time: 52.57, best: 19778, avg: 22471
iter: 600, time: 62.87, best: 19778, avg: 22044
iter: 700, time: 72.96, best: 19778, avg: 21243
iter: 800, time: 84.23, best: 19778, avg: 22030
iter: 900, time: 94.47, best: 19778, avg: 21180
iter: 1000, time: 104.06, best: 19778, avg: 22588
iter: 1100, time: 113.81, best: 19778, avg: 22921
iter: 1200, time: 124.75, best: 19778, avg: 23015
iter: 1300, time: 133.5, best: 19778, avg: 22842
iter: 1400, time: 143.81, best: 19778, avg: 23147
iter: 1500, time: 154.24, best: 19778, avg: 23186
iter: 1600, time: 165.55, best: 19778, avg: 22125
iter: 1700, time: 178.52, best: 19778, avg: 23687
iter: 1800, time: 188.67, best: 19778, avg: 23072
iter: 1900, time: 198.48, best: 19778, avg: 22199
iter: 2000, time: 209.59,